In [31]:
import glob, re, os, shutil, random
import pandas as pd
from pathlib import Path
import numpy as np

In [32]:
#############################################
######### Split label file E/I, ... #########
#############################################

TARGET_LIST = ['E/I', 'F/T', 'J/P', 'N/S']
for TARGET in TARGET_LIST:
    df = pd.read_csv("data\\label.csv", index_col= "No.")
    df.drop(df.columns[-3:], inplace=True, axis = 1)
    df.rename(columns={df.columns[0]:'MBTI'}, inplace=True)

    trait = TARGET.split('/')
    df2 = pd.DataFrame(columns=['filename', f'{trait[0]}', f'{trait[1]}'])

    os.makedirs(f'data\\{trait[0]}_{trait[1]}\\{trait[0]}', exist_ok=True)
    os.makedirs(f'data\\{trait[0]}_{trait[1]}\\{trait[1]}', exist_ok=True)

    for file in glob.glob("data\\images\\*.jpg"):
            file_id = file.replace('_', '\\').split('\\')[2]
            personality = df.iloc[int(file_id)][TARGET].strip(' ')

            new_row = {
                    'filename' : file, 
                    f'{trait[0]}' : int(personality == trait[0]),
                    f'{trait[1]}' : int(personality == trait[1])
            }
            df2 = df2.append(new_row, ignore_index=True)
        
            SOURCE_PATH = f'{file}'
            TARGET_PATH = f'data\\{trait[0]}_{trait[1]}\\{personality}'

            shutil.copy(SOURCE_PATH, TARGET_PATH)

    df2.to_csv(f'data\\label_{trait[0]}_{trait[1]}.csv', index=False)